In [1]:
# Install pyspark on Jupyter Notebook
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import Library
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [3]:
#Create a spark session
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

23/03/19 00:27:42 WARN Utils: Your hostname, rickyh-My resolves to a loopback address: 127.0.1.1; using 192.168.0.104 instead (on interface wlp1s0)
23/03/19 00:27:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/19 00:27:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Schema of data
df = spark.read.option("header", "true").option("inferSchema", "true")\
    .csv('/home/rickyh/DE_Learn/homework_spark/input/yellow_tripdata_2021-01.csv.gz')
print("Here is our inferred schema:")
df.printSchema()

Here is our inferred schema:
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [5]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|              1|          2.1|         1|                 N|         142|          43|           2|        8.0|  3.0|    0.5|       0.0|         0.0|                  0.3

In [6]:
df.createOrReplaceTempView("tripData")

In [7]:
# Query SQL
sqlDF = spark.sql("SELECT * FROM \
tripData where PULocationID is not null and DOLocationID is not null and VendorID is not null \
and tpep_pickup_datetime >= '2021-01-01' and tpep_pickup_datetime <= '2021-01-10'\
ORDER BY VendorID, tpep_pickup_datetime ASC")
sqlDF.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:00:18|  2021-01-01 00:07:17|              2|          0.9|         1|                 N|         170|         233|           2|        6.0|  3.5|    0.5|       0.0|         0.0|                  0.3

In [8]:
sqlDF.write.parquet("/home/rickyh/DE_Learn/homework_spark/output/spark_write_parquet.parquet")

In [9]:
df_parquet = spark.read.parquet("/home/rickyh/DE_Learn/homework_spark/output/spark_write_parquet.parquet")
df_parquet.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:00:18|  2021-01-01 00:07:17|              2|          0.9|         1|                 N|         170|         233|           2|        6.0|  3.5|    0.5|       0.0|         0.0|                  0.3

In [16]:
sqlTripData = spark.sql("SELECT VendorID, PUlocationID, DOlocationID, trip_distance, tip_amount, total_amount \
FROM tripData WHERE VendorID IS NOT NULL AND PULocationID IS NOT NULL and DOLocationID is NOT NULL AND tpep_pickup_datetime >= ' 2021-01-01' \
AND tpep_pickup_datetime <=' 2021-01-10'\
ORDER BY VendorID")
sqlTripData.show()

+--------+------------+------------+-------------+----------+------------+
|VendorID|PUlocationID|DOlocationID|trip_distance|tip_amount|total_amount|
+--------+------------+------------+-------------+----------+------------+
|       1|         249|         137|          2.2|       0.0|        12.8|
|       1|         162|          75|          3.0|      2.85|       17.15|
|       1|         107|         163|          3.6|      10.0|        30.8|
|       1|         238|         151|          0.2|       0.0|         4.3|
|       1|         132|           3|         19.1|       0.0|       59.42|
|       1|         138|         132|         10.6|      6.05|       36.35|
|       1|          48|         226|          5.4|       0.0|        23.8|
|       1|         224|          68|          1.6|      2.35|       14.15|
|       1|         132|         226|         14.3|       0.0|        40.3|
|       1|          90|          40|          5.7|       0.0|        21.8|
|       1|         138|  